In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

C:\Users\weezs\Anaconda3\envs\minjoo\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\weezs\Anaconda3\envs\minjoo\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('data/glass.csv')
df[:5]

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,Type
0,1.51793,12.79,3.50,1.12,73.03,0.64,8.77,0.0,0.00,'build wind float'
1,1.51643,12.16,3.52,1.35,72.89,0.57,8.53,0.0,0.00,'vehic wind float'
2,1.51793,13.21,3.48,1.41,72.64,0.59,8.43,0.0,0.00,'build wind float'
3,1.51299,14.40,1.74,1.54,74.55,0.00,7.59,0.0,0.00,tableware
4,1.53393,12.30,0.00,1.00,70.16,0.12,16.19,0.0,0.24,'build wind non-float'


In [5]:
X = df.values[:,:-1]
y = df.values[:,-1]

In [6]:
# Possible values: 'euclidean', 'manhattan', 'minkowski', 'chebyshev' 
# clf = KNeighborsClassifier(n_neighbors=5, metric='manhattan')
clf = KNeighborsClassifier(n_neighbors=5, p=1)

In [7]:
# shuffle :: 데이터를 나눌때 전체를 한번 섞고나서 나눈다.
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [8]:
print(cross_val_score(clf, X, y, cv=cv).mean())

0.6666666666666667


Attributes of Mixed Type

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OrdinalEncoder

In [12]:
df = pd.read_csv('data/weather.numeric.csv')
df[:5]

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes


In [13]:
X = df.values[:, :-1]
y = df.values[:, -1]
cv = KFold(n_splits=10, shuffle=True, random_state=0)

In [14]:
encoder_ordinal = OrdinalEncoder([['sunny','overcast','rainy']])
encoder_ordinal = encoder_ordinal.fit_transform(X[:,[0]])
X[:,[0]] = encoder_ordinal
print(X)

[[0.0 85 85 False]
 [0.0 80 90 True]
 [1.0 83 86 False]
 [2.0 70 96 False]
 [2.0 68 80 False]
 [2.0 65 70 True]
 [1.0 64 65 True]
 [0.0 72 95 False]
 [0.0 69 70 False]
 [2.0 75 80 False]
 [0.0 75 70 True]
 [1.0 72 90 True]
 [1.0 81 75 False]
 [2.0 71 91 True]]


In [17]:
df.values[:,0]

array(['sunny', 'sunny', 'overcast', 'rainy', 'rainy', 'rainy',
       'overcast', 'sunny', 'sunny', 'rainy', 'sunny', 'overcast',
       'overcast', 'rainy'], dtype=object)

In [18]:
pd.factorize(df.values[:,0])

(array([0, 0, 1, 2, 2, 2, 1, 0, 0, 2, 0, 1, 1, 2], dtype=int64),
 array(['sunny', 'overcast', 'rainy'], dtype=object))

In [19]:
# 순서를 지정하지 않으면? 알아서 순서대로..
encoder_ordinal = OrdinalEncoder()
encoder_ordinal = encoder_ordinal.fit_transform(X[:,[3]])
X[:,[3]] = encoder_ordinal
print(X)

[[0.0 85 85 0.0]
 [0.0 80 90 1.0]
 [1.0 83 86 0.0]
 [2.0 70 96 0.0]
 [2.0 68 80 0.0]
 [2.0 65 70 1.0]
 [1.0 64 65 1.0]
 [0.0 72 95 0.0]
 [0.0 69 70 0.0]
 [2.0 75 80 0.0]
 [0.0 75 70 1.0]
 [1.0 72 90 1.0]
 [1.0 81 75 0.0]
 [2.0 71 91 1.0]]


In [29]:
num_att =  4
types = ('ordinal','numeric','numeric','nominal')

In [30]:
maximum = np.empty(num_att)
minimum = np.empty(num_att)

for i in range(num_att):
    if types[i] == 'numeric':
        maximum[i] = X[:,i].max()
        minimum[i] = X[:,i].min()

In [31]:
# nominal attributes
def nominal_distance(a,b):
    if a == b:
        return 0
    else:
        return 1

In [32]:
# numeric attributes
def numeric_distance(a,b,minimum,maximum):
    return np.abs(a-b)/(maximum-minimum)

In [33]:
# ordinal attributes
# columns 갯수가 3개이므로 분모에 2를 대입하였고, 이에따라 0~1값으로 norm된다.
def ordinal_to_numeric(s):
    return s / 2

def ordinal_distance(a,b):
    return numeric_distance(ordinal_to_numeric(a),ordinal_to_numeric(b),0,1)

In [34]:
def mixed_dist(a,b):
    d = np.empty(num_att)
    for i in range(num_att):
        if types[i] == 'ordinal':
            d[i] = ordinal_distance(a[i],b[i])
        elif types[i] == 'numeric':
            d[i] = numeric_distance(a[i],b[i],minimum[i],maximum[i])
        else:
            d[i] = nominal_distance(a[i],b[i])
    return d.mean()

In [35]:
clf = KNeighborsClassifier(n_neighbors=5, metric=mixed_dist)
print(cross_val_score(clf,X,y,cv=cv).mean())

0.55


In [36]:
mixed_dist(X[0,:],X[1,:])

0.3498463901689708